# Desarrollo de un sistema de diagnóstico de enfermedades en hojas de tomate mediante PlantCV y modelos de aprendizaje profundo

El conjunto de datos contiene más de 20.000 imágenes de hojas de tomate divididas en 11 clases (10 tipos de enfermedades y 1 clase de hojas sanas). Las imágenes se recogieron tanto en entornos de laboratorio como en condiciones naturales. El objetivo de este proyecto es crear un modelo capaz de detectar las enfermedades de las hojas del tomate y que pueda utilizarse en múltiples plataformas.

## Carga de datos de kaggle

In [ ]:
%%capture
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
import io
from google.colab import files

seed = 42  # Semilla aleatoria arbitraria y constante a incluir en los algoritmos estocásticos para que los experimentos sean siempre reproducibles por el profesor.

def upload_files (index_fields=None):
  uploaded = files.upload()

In [ ]:
# Seleccionar el API Token personal previamente descargado (fichero kaggle.json)
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir my_dataset

In [ ]:
%%capture
!unzip tomato.zip -d my_dataset

In [ ]:
!ls my_dataset/

In [ ]:
!ls my_dataset/train

In [ ]:
!ls my_dataset/train/Bacterial_spot

Hasta ahora se ha descargado el dataset de kaggle https://www.kaggle.com/datasets/tr1gg3rtrash/yoga-posture-dataset en un zip y se ha descomprimido en la carpeta my_dataset/. Se ha comprobado que estedirectorio contiene otros directorios con ciertas imágenes. Parece que hay distintos tipos de imágenes como .png o .jpeg.

## Exploración de directorio

In [ ]:
%%capture
!pip install plantcv

In [ ]:
import os
import random
import shutil
from collections import Counter
import matplotlib.pyplot as plt

from plantcv import plantcv as pcv

In [ ]:
def explore_directory(dir, top_k):

    size_counter = Counter()
    example_paths = {}
    error_imgs = 0

    if os.path.exists(dir): # Verifica si existe el directorio

        for subdir in os.listdir(dir): # Recorre subdirectorios:
            subdirPath = os.path.join(dir, subdir)
            if os.path.isdir(subdirPath): # Comprueba que está en un directorio
                for file in os.listdir(subdirPath):
                    filePath = os.path.join(subdirPath, file) # Ruta completa de cada archivo
                    if os.path.isfile(filePath): # Verifica si es un archivo
                        img_path = os.path.join(filePath)
                        try:
                            img, _, _ = pcv.readimage(filename=img_path)
                            height, width = img.shape[:2]
                            size = (width, height)
                            size_counter[size] += 1
                            if size not in example_paths:
                                example_paths[size] = img_path
                        except:
                            error_imgs += 1
                            continue  # skip corrupted or unreadable images
                    else:
                      print("No es un archivo")
            else:
                print("No es un directorio")
    else:
        print("El directorio especificado no existe")


    # Mostrar los tamaños más comunes
    most_common_sizes = size_counter.most_common(top_k)
    print(f"Tamaño de imagen más común (Top {top_k}):")
    for size, count in most_common_sizes:
        print(f"  {size} → {count} imágenes")

    print(f"\nNúmero de tamaños de imágenes diferentes: {len(size_counter)}")

    print(f"\nNúmero de imágenes ilegibles: {error_imgs}")

In [ ]:
explore_directory('my_dataset/train/', 5)

In [ ]:
def plot_random_images_per_class(dir):

  classes = sorted(os.listdir(dir))# Listar subdirectorios (clases)

  plt.figure(figsize=(15, 8))  # Crear figura

  for i, cls in enumerate(classes):
    cls_dir = os.path.join(dir, cls)
    if os.path.isdir(cls_dir): # Comprueba que es en un directorio
      images = [f for f in os.listdir(cls_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))] # Filtra archivos de tipo .jpg, .jpeg, .png (imagen)
      img_path = os.path.join(cls_dir, random.choice(images)) # Elegir una imagen aleatoria
      img, _, _ = pcv.readimage(filename=img_path) # Leer imagen

      # Dibujar en un subplot
      plt.subplot(3, 4, i+1)  # ajusta según nº de clases
      plt.imshow(img)
      plt.title(cls, fontsize=14)
      plt.axis("off")

  plt.tight_layout()
  plt.show()

Se observa la variedad de imágenes que existen. Unas más oscuras, otras con más brillo. Algunas con fundo blanco, otras con fondo de color o incluso con más elementos de fondo no solo un fondo plano. Incluso algunas están giradas.

## Preprocesamiento

In [ ]:
%%capture
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import TFSMLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras import regularizers

En el conjunto de datos no hay ninguna carpeta de pruebas (test). Por lo tanto, tenemos que mover algunas imágenes de la carpeta de entrenamiento a una nueva carpeta llamada test. Utilizando una proporción del 20%.

In [ ]:
test_dir = 'my_dataset/test/'

# Proporción de datos para el conjunto de test
test_ratio = 0.2

# Semilla para aleatorización
random.seed(42)

# Bucle para recorrer cada clase (directorio) del directorio de entrenamiento
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        # Crear directorio test/<class> si no existe
        target_class_path = os.path.join(test_dir, class_name)
        os.makedirs(target_class_path, exist_ok=True)

        # Lista de imágenes en directorio de cada clase
        images = [
            f for f in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, f))
        ]
        random.shuffle(images)  # Aleatorizar imágenes

        # Cuántas imágenes pasan a conjunto de test
        n_test = int(len(images) * test_ratio)
        test_images = images[:n_test]

        # Mueve la imagen al directorio de test
        for img_name in test_images:
            src_path = os.path.join(class_path, img_name)
            dst_path = os.path.join(target_class_path, img_name)
            shutil.move(src_path, dst_path)

    else:
        print('No es un directorio:', class_path)

Se forma el conjunto de datos de entrenamiento y validación a partir de los directorios donde se encuentran las imágenes. Éstas se convertirán a tamaño 256x256 píxeles (ya que es el tamaño más común) y se procesarán en lotes de 32. La función image_dataset_from_directory lee automáticamente las imágenes de los subdirectorios y reconoce los nombres éstos como etiquetas de clase.

In [ ]:
train_dir = "my_dataset/train/"
valid_dir = "my_dataset/valid/"

BATCH_SIZE = 32
IMG_SIZE = (256, 256)

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)